In [5]:
import random
import pprint
import sys
import json
from keras_frcnn import config

sys.setrecursionlimit(40000)

C = config.Config()
C.num_rois = 2

C.use_vertical_flips = True

#from keras_frcnn.pascal_voc_parser import get_data
from keras_frcnn.simple_parser import get_data

from keras_frcnn import data_generators
from keras import backend as K

from keras_frcnn import resnet as nn
from keras.optimizers import Adam, SGD
from keras.layers import Input
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_frcnn import losses
from keras.callbacks import ReduceLROnPlateau

all_imgs, classes_count, class_mapping = get_data('/home/mh/ws/fish_challenge/input/im2txt.txt')

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (None, None, 3)

img_input = Input(shape=input_shape_img)

roi_input = Input(shape=(C.num_rois, 4))

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = nn.rpn(shared_layers, num_anchors)

# the classifier is build on top of the base layers + the ROI pooling layer + extra layers
classifier = nn.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count)+1, trainable=True)

# define the full model
model = Model([img_input, roi_input], rpn + classifier)


try:
	print('loading weights from ', C.base_net_weights)
	model.load_weights('/home/mh/.keras/models'+resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5,by_name=True)
except:
	print(('Could not load pretrained model weights. Weights can be found at {} and {}'.format(
		'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels_notop.h5',
        'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
	        )))

sgd = SGD(lr=1e-2, decay=1e-4, momentum=0.88, nesterov=False)
adam = Adam(1e-5, decay=0.0)

print('loaded weights')
model.compile(optimizer=adam, loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors), losses.class_loss_cls, losses.class_loss_regr(C.num_rois,len(classes_count)-1)], metrics={'dense_class_{}_loss'.format(len(classes_count)): 'accuracy'})

callbacks = [EarlyStopping(monitor='val_loss', patience=20, verbose=0),
				ModelCheckpoint(C.model_path, monitor='val_loss', save_best_only=True, verbose=0),
			 ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1)]


model.load_weights('/home/mh/ws/fish_challenge/input/'+'models/frcnn_1_7epochs.h5')



Parsing annotation files


/home/mh/ws/fish_challenge/Fishy-master/notbook/keras_frcnn/resnet.py:123: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, shortcut], mode='sum')
/home/mh/.local/lib/python3.5/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/mh/ws/fish_challenge/Fishy-master/notbook/keras_frcnn/resnet.py:50: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, input_tensor], mode='sum')
/home/mh/ws/fish_challenge/Fishy-master/notbook/keras_frcnn/resnet.py:161: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, 

loading weights from  resnet50_weights_tf_dim_ordering_tf_kernels.h5
Could not load pretrained model weights. Weights can be found at https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels_notop.h5 and https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
loaded weights
